In [ ]:
import gurobipy as gp
import math
import pandas as pd

In [ ]:
IP = 0
MIP = 1
LP = 2


# Model Fermat's Factorization method
def model_fermat(N, model_type, m):
    # find smallest integer k such that N <= 2^k
    k = 0
    power = 1
    while power < N:
        power *= 2
        k += 1
        
        
    # Add variables
    if model_type == IP:
        A = m.addVar(vtype=gp.GRB.INTEGER, name="A")
        A_bits = m.addVars(1, k, vtype=gp.GRB.BINARY, name="A_bits")
        A_square = m.addVar(vtype=gp.GRB.INTEGER, name="A_square")
        A_square_products = m.addVars(1, k, vtype=gp.GRB.INTEGER, name="A_square_products")
        
        B = m.addVar(vtype=gp.GRB.INTEGER, name="B")
        B_bits = m.addVars(1, k, vtype=gp.GRB.BINARY, name="B_bits")
        B_square = m.addVar(vtype=gp.GRB.INTEGER, name="B_square")
        B_square_products = m.addVars(1, k, vtype=gp.GRB.INTEGER, name="B_square_products")
        
    elif model_type == MIP or model_type == LP:
        A = m.addVar(vtype=gp.GRB.CONTINUOUS, name="A")
        A_square = m.addVar(vtype=gp.GRB.CONTINUOUS, name="A_square")
        A_square_products = m.addVars(1, k, vtype=gp.GRB.CONTINUOUS, name="A_square_products")
        
        B = m.addVar(vtype=gp.GRB.CONTINUOUS, name="B")
        B_square = m.addVar(vtype=gp.GRB.CONTINUOUS, name="B_square")
        B_square_products = m.addVars(1, k, vtype=gp.GRB.CONTINUOUS, name="B_square_products")
        
        if model_type == MIP:
            A_bits = m.addVars(1, k, vtype=gp.GRB.BINARY, name="A_bits")
            B_bits = m.addVars(1, k, vtype=gp.GRB.BINARY, name="B_bits")
        else:
            A_bits = m.addVars(1, k, vtype=gp.GRB.CONTINUOUS, name="A_bits")
            B_bits = m.addVars(1, k, vtype=gp.GRB.CONTINUOUS, name="B_bits")

        
    # Set objective
    m.setObjective(0, gp.GRB.MAXIMIZE)

    
    # Add constraints
    
    # add main constraint
    m.addConstr(N == A_square - B_square)
    
    # add constraints for A and A_square
    power = 1
    A_sum = 0
    A_square_sum = 0
    for i in range(k):
        A_sum += power * A_bits[0,i]
        A_square_sum += power * A_square_products[0,i]
        power *= 2
    
    m.addConstr(A == A_sum)
    m.addConstr(A_square == A_square_sum)
    
    for i in range(k):
        m.addConstr(A_square_products[0,i] <= N * A_bits[0,i])
        m.addConstr(A_square_products[0,i] <= A)
        m.addConstr(A - N*(1 - A_bits[0,i]) <= A_square_products[0,i])
        m.addConstr(0 <= A_square_products[0,i])
    
    # add constraints for B and B_square
    power = 1
    B_sum = 0
    B_square_sum = 0
    for i in range(k):
        B_sum += power * B_bits[0,i]
        B_square_sum += power * B_square_products[0,i]
        power *= 2
    
    m.addConstr(B == B_sum)
    m.addConstr(B_square == B_square_sum)
    
    for i in range(k):
        m.addConstr(B_square_products[0,i] <= N * B_bits[0,i])
        m.addConstr(B_square_products[0,i] <= B)
        m.addConstr(B - N*(1 - B_bits[0,i]) <= B_square_products[0,i])
        m.addConstr(0 <= B_square_products[0,i])
        
    # add constraint to prevent trivial factorization
    m.addConstr(B <= A-2)
    
    
    # extra constraints for LP relaxation
    if model_type == LP:
        for i in range(k):
            m.addConstr(0 <= A_bits[0,i])
            m.addConstr(A_bits[0,i] <= 1)
            m.addConstr(0 <= B_bits[0,i])
            m.addConstr(B_bits[0,i] <= 1)
    

In [ ]:
current_type = IP
start = 1000000
end = start + 300
data = []

for N in range(start, end+1):
    if N % 2 == 0:
        continue
    
    m = gp.Model()
    m.Params.LogToConsole = 0
    m.Params.Threads = 2
    
    model_fermat(N, current_type, m)
    
    for i in range(10):
        m.reset(1)
        m.optimize()

        if m.status == gp.GRB.OPTIMAL:
            # get values of A^2 and B^2
            A_square = m.getVarByName("A_square").x
            B_square = m.getVarByName("B_square").x
            
            correct = 0
            if A_square.is_integer() and B_square.is_integer() \
               and math.sqrt(A_square).is_integer() and math.sqrt(B_square).is_integer() \
               and N == A_square - B_square:
                correct = 1


            data.append([N, A_square, B_square, correct, \
                         m.NumVars, m.NumIntVars, m.NumBinVars, m.NumConstrs, \
                         m.NodeCount, m.IterCount, m.runtime, m.IsMIP])
        elif m.status == gp.GRB.INFEASIBLE:
            data.append([N, "INFEASIBLE", "INFEASIBLE", "N/A", \
                         m.NumVars, m.NumIntVars, m.NumBinVars, m.NumConstrs, \
                         m.NodeCount, m.IterCount, m.runtime, m.IsMIP])

In [ ]:
df = pd.DataFrame(data, columns=["N", "A^2", "B^2", "Is Correct", \
                                 "Num Vars", "Num Integer Vars", "Num Binary Vars", "Num Constraints", \
                                 "Node Count", "Simplex Iterations Count", "Runtime", "Is MIP"])

In [ ]:
print(df)